In [1]:
using ITensors
using PyCall

np = pyimport("numpy")

using Plots
pyplot()

Plots.PyPlotBackend()

<h1> TDVP Routine </h1>

In [2]:
using TimeEvoMPS

<h1> Define the hamiltonian </h1>

In [3]:
h = 0.
J = 1

Lx = 6
Ly = 6

L  = Lx*Ly

sites = siteinds("S=1/2",L)
psi0 = randomMPS(sites)
ampo = OpSum()

for i=1:L
    val = mod(i,Ly)
    if mod(i,Ly) != 0
        if i != L
            ampo += -4*J,"Sz",i,"Sz",i+1
        end
    end
    if i <= (Lx-1)*Ly
        val = i + Ly
        ampo += -4*J,"Sz",i,"Sz",i+Ly
    end
    ampo += -2*h,"Sx",i
end

H = MPO(ampo,sites)

sweeps = Sweeps(7)
setmaxdim!(sweeps, 10,20,100,200,300,300,400)
setnoise!(sweeps,1e-5,1e-6,1e-8,1e-8,1e-10,0.0,0.0)
setcutoff!(sweeps, 1E-15)
#@show sweeps

#z_observer = DMRGObserver(["Sz"],sites,energy_tol=1E-12)

energy, psi = dmrg(H, psi0, sweeps)#,observer=Sz_observer)
println("Final energy = $energy")

After sweep 1 energy=-59.999927122960 maxlinkdim=10 maxerr=1.91E-07 time=16.348
After sweep 2 energy=-59.999999999188 maxlinkdim=10 maxerr=1.55E-14 time=0.079
After sweep 3 energy=-60.000000000000 maxlinkdim=1 maxerr=3.96E-16 time=0.075
After sweep 4 energy=-60.000000000000 maxlinkdim=1 maxerr=5.79E-38 time=0.018
After sweep 5 energy=-60.000000000000 maxlinkdim=1 maxerr=5.79E-40 time=0.020
After sweep 6 energy=-60.000000000000 maxlinkdim=1 maxerr=8.92E-71 time=4.407
After sweep 7 energy=-60.000000000000 maxlinkdim=1 maxerr=3.65E-114 time=0.013
Final energy = -60.00000000000003


In [ ]:
psi = productMPS(sites, fill("↑",L))
T  = 0.5
dt = 0.05
cb = LocalMeasurementCallback(["Sz"], sites, T)
tdvp!(psi,H,dt,T,callback=cb)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\asant\.julia\packages\ProgressMeter\Vf8un\src\ProgressMeter.jl:620
Evolving state...  50%|█████████████████                |  ETA: 0:03:52
  t:             0.25
  dt step time:  201.988
  Max bond-dim:  1024